### Processing TCGA-BRCA MRI exams

The following script:
* processes DCE-MRI exams in the TCGA-BRCA dataset:
    1. figures out a pre-contrast and two post-contrast sequences
    2. resamples them to the same anisotropic spacing of [0.7142857, 0.7142857, 1.2]
    3. reorients them to LPS orientation
    4. saves them to a nifti (.nii) format
* saves breast-level labels for the dataset, i.e. whether there is breast cancer in left and right breasts

Please note that this notebook will pre-process most of the DCE-MRI exams in the data set, but not all of them.

In [ ]:
import os
import yaml
import re
from typing import List
import pathlib
import numpy as np
from tqdm import tqdm
import SimpleITK as sitk
import pandas as pd

### 0. Helper functions - please run them once

In [ ]:
def get_volume_orientation(volume: sitk.Image) -> str:
    orient_filter = sitk.DICOMOrientImageFilter()
    cosines = volume.GetDirection()
    return orient_filter.GetOrientationFromDirectionCosines(cosines)

def resample_volume(volume: sitk.Image,
                    new_spacing: List[float],
                    interpolator=sitk.sitkLinear) -> sitk.Image:
    """
    Change spacing
    :param volume: Input image to be resampled
    :param new_spacing: For our volumes this should be a 3-item list
                        i.e. (1.0, 1.0, 1.0)
    :param interpolator: Which interpolator to use. Preferred Linear,
                        for segmentation nearest neighbor
    """
    new_size = [int(round(osz*ospc/nspc)) 
                for osz, ospc, nspc 
                in zip(volume.GetSize(), volume.GetSpacing(), new_spacing)]
    minimum_value = sitk.GetArrayFromImage(volume).min()
    return sitk.Resample(volume,
                         new_size,
                         sitk.Transform(),
                         interpolator,
                         volume.GetOrigin(),
                         new_spacing,
                         volume.GetDirection(),
                         int(minimum_value),
                         volume.GetPixelID())

def reorient_to_lps(volume: sitk.Image,
                    verbose: bool = False) -> sitk.Image:
    """
    If volume is not in the LPS orientation, reorient to LPS
    """
    current_orientation = get_volume_orientation(volume)
    if current_orientation != "LPS":
        if verbose:
            print(f"Reorienting {current_orientation} to LPS")
            reorient_time = time.time()
        orientation_filter = sitk.DICOMOrientImageFilter()
        orientation_filter.SetDesiredCoordinateOrientation("LPS")
        volume = orientation_filter.Execute(volume)
        if verbose:
            reorient_time_end = time.time()
            print("Reotienting time:", reorient_time_end - reorient_time)
    return volume

In [ ]:
def process_multivolume(studydir):
    subvolumes = {}
    
    # This flag should be set to True if processing is complete
    # and subvolumes(dict) is properly propagated
    subvolumes_generated = False
    
    # Initially, check filenames for format D-DDD.dcm (D=digit)
    # a lot of studies will have sub-volumes separated by this format
    # e.g. 1-DDD.dcm for pre-contrast, 2-DDD.dcm for first post-contrast,
    # 3-DDD.dcm for second post-contrast etc.
    for dn in sorted(os.listdir(studydir)):
        if not re.match(r"\d-\d\d\d.dcm", dn):
            print("\tNot matched regexp:", dn)
            break
        else:   # matched
            if dn[0] not in subvolumes:
                subvolumes[dn[0]] = []
            subvolumes[dn[0]].append(dn)
    
    # If we were able to generate initial subvolumes
    # using filenames
    if len(subvolumes) > 0:       
        # Option one: There are multiple subvolumes (<2), each should have
        # an even number of images
        if len(subvolumes) > 1:
            images_num_to_match = None
            failed_check = False
            for k, v in subvolumes.items():
                if images_num_to_match is None:
                    images_num_to_match = len(v)
                else:
                    if len(v) != images_num_to_match:
                        print("\tOption 1 failed.")
                        failed_check = True
            if not failed_check:
                subvolumes_generated = True
                print("Option 1 successful.")
                return subvolumes
            else:
                print("\tOption 1 failed!")
        
        # Option two: There is first subvolume format (1-DDD.dcm) for precontrast
        # and then second subvolume (2-DDD.dcm) for postcontrast.
        if len(subvolumes) == 2:
            print("\t Running failed_check")
            new_subvolumes = {}
            potential_num_prec = subvolumes["1"]
            new_subvolumes["1"] = subvolumes["1"]
            print("\t division modulo:", len(subvolumes["2"]) % len(subvolumes["1"]))
            if len(subvolumes["2"]) % len(subvolumes["1"]) == 0:
                failed_check = False  # match!
                postcontrast_num_subvolumes = len(subvolumes["2"]) // len(subvolumes["1"])
                for i in range(2, 2+postcontrast_num_subvolumes):
                    new_subvolumes[str(i)] = [f"2-{i:03}.dcm" for i in range((i-1)*len(subvolumes["1"])+1,i*len(subvolumes["1"])+1)]
                print(f"\tNew subvolumes:\n\t{new_subvolumes}")
                subvolumes_generated = True
            if subvolumes_generated:
                print("Option 2 successful.")
                return new_subvolumes
            else:
                raise ValueError("Subvolumes=2 but failed to process.")
    
    # Check for TriggerTime policy
    # this is slow because we have to read each DICOM file to check for TriggerTime tag
    if len(subvolumes) <= 1:
        print("\tTrying trigger time policy...")
        
        tt = {}  # dictionary of structure: key is trigger time, value is a list of filenames
        
        for dn in sorted(os.listdir(studydir)):
            ds = pydicom.dcmread(os.path.join(studydir, dn))            

            if ds.TriggerTime not in tt:
                tt[ds.TriggerTime] = []
            tt[ds.TriggerTime].append(dn)

        # Check that all subvolumes generated from trigger time
        # have the same number of images
        check_failed = False
        num_expected = None
        for k, v in tt.items():
            if num_expected is None:
                num_expected = len(v)
            else:
                if len(v) != num_expected:
                    check_failed = True
        if check_failed:
            print("\tTriggerTime policy failed.")
        else:
            print("\tTriggerTime successful.")
            tt = dict(sorted(tt.items()))  # sort by key (trigger time)
            
            k_i = 1
            tt_keys = list(tt.keys())
            for k in tt_keys:
                tt[str(k_i)] = tt.pop(k)
                k_i += 1
            
            return tt
    
    # If the function hasn't returned subvolume dict yet,
    # then we failed and nothing has been processed.
    
    print("\tMulti-volume processing failed!")

In [ ]:
def get_volume_given_filepaths(filepaths: list, reorient_resample=True):
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(filepaths)
    volume = reader.Execute()
    
    if reorient_resample:
        print("Image size before:", volume.GetSize())
        volume = reorient_to_lps(volume)
        volume = resample_volume(volume, [0.7142857, 0.7142857, 1.2])
        print("Image size after:", volume.GetSize())
    
    return volume

In [ ]:
def read_study_dir(studydir, resample=True):
    print("READING: ", studydir)
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(studydir)
    #print(dicom_names)
    reader.SetFileNames(dicom_names)

    series_volume = reader.Execute()
    
    print("Image size before:", series_volume.GetSize())
    
    # Reorient to LPS
    series_volume = reorient_to_lps(series_volume)
    print("Image size post reorient:", series_volume.GetSize())
    
    # Resample to new spacing
    if resample:
        series_volume = resample_volume(volume=series_volume, new_spacing=[0.7142857, 0.7142857, 1.2])
    
    size = series_volume.GetSize()
    #print("Image size after:", size[0], size[1], size[2])
    
    return series_volume

In [ ]:
def match_serie_dirnames(subject, study_id, pre_post_dict, basedir):
    """
    This returns a dictionary in structure:
        t1pre: [dirnames where all pre-c images are],
        t1c1: [dirnames where all 1st post-c images are],
        t1c2: [dirnames where all 2st post-c images are],
    """
    study_dicom_dir = os.path.join(basedir, subject, study_id)
    
    study_yaml = pre_post_dict['subjects'][subject]['studies'][study_id]
    series_yaml = study_yaml['series']
    
    serie_dirname_dict = {
        "t1pre": [],
        "t1c1": [],
        "t1c2": []
    }
    serie_dir_names = os.listdir(study_dicom_dir)
   
    for serie_num, serie_type in series_yaml.items():
        if serie_type in serie_dirname_dict:
            # 1. find the dirname
            for serie_dn in serie_dir_names:
                if serie_dn.startswith(f"{str(serie_num)}.0"):
                    # 2. add to the dict
                    serie_dirname_dict[serie_type].append(serie_dn)
    
    return serie_dirname_dict

In [ ]:
def generate_volumes_split_bilateral(subject, study_id, serie_dirnames, basedir, reorient_resample=True):   
    sitk_dict = {}  # this will keep simpleitk images
    
    for serie_type in ['t1pre', 't1c1', 't1c2']:
        arrs = []

        # assumes that spacing and direction are the same for both volumes
        # NOTE: Maybe there should be a sanity check here that both are same/similar
        spacing = None
        direction = None

        for single_dir in serie_dirnames[serie_type]:
            serie_full_path = os.path.join(basedir, subject, study_id, single_dir)
            serie_files = [os.path.join(serie_full_path, x) for x in sorted(os.listdir(serie_full_path))]

            vol = get_volume_given_filepaths(serie_files, reorient_resample=False)

            if spacing is None:
                spacing = vol.GetSpacing()
                direction = vol.GetDirection()

            arrs.append(sitk.GetArrayFromImage(vol))

        # combine both to numpy array
        stacked = np.vstack(arrs)

        # convert back to simpleitk
        stacked_sitk = sitk.GetImageFromArray(stacked)

        stacked_sitk.SetSpacing(spacing)
        stacked_sitk.SetDirection(direction)
        
        if reorient_resample:
            stacked_sitk = reorient_to_lps(stacked_sitk)
            stacked_sitk = resample_volume(stacked_sitk, [0.7142857, 0.7142857, 1.2])
        
        sitk_dict[serie_type] = stacked_sitk
        
    return sitk_dict

In [ ]:
def preprocess_tcga_brca(pre_post_dict, data_basedir, save_basedir, dry_run=False, verbose=False):
    processed_studies = 0
    study_paths_dict = {}
    
    if dry_run:
        print("Dry run, no actual processing will be done.")
    with tqdm(total=139) as pbar:
        for subject in pre_post_dict['subjects']:
            pbar.update(1)
            pbar.set_description(f"P: {subject}")
            
            # Get studies for subject
            try:
                s_studies = pre_post_dict['subjects'][subject]['studies']        
            except:
                print(f"[{subject}] has no correct studies")
                continue
                
            # Iterate over all expected studies for this subject
            for study_id in s_studies:
                pbar.set_description(f"P: {subject}, S: {study_id}")
                
                if os.path.exists(os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"))):
                    # Skip if already saved
                    if verbose:
                        print(f"[{subject}/{study_id}] already saved -- skipping")
                    continue

                study = s_studies[study_id]
                
                # Get laterality type
                try:
                    lat_type = study['lateralityType']
                except:
                    print(f"[{subject}/{study_id}] does not have lateralityType value.")
                    continue
                if lat_type == 'unknown':
                    print(f"[{subject}/{study_id}] has unknown laterality. Excluding.")
                    continue
                
                # Get info about all series for current study
                series = study['series']
                
                # PROCESSING OF EACH SERIE DEPENDING ON TYPE OF LATERALITY IN IMAGES
                # (This is where the actual processing happens)
                if verbose:
                    print(f"[{subject}/{study_id}]: laterality type {lat_type}")
                if dry_run:
                    continue
                
                if lat_type == 'splitBilateral':
                    print(f"Lat type {lat_type} is not handled by this script.")
                    continue  # Continue to next series
                elif lat_type == 'bilateral':
                    check_pre = False
                    check_c1 = False
                    check_c2 = False

                    for serie_id, serie_type in series.items():
                        #print(serie_id, serie_type)
                        if serie_type == 't1pre':
                            check_pre = serie_id
                        elif serie_type == 't1c1':
                            check_c1 = serie_id
                        elif serie_type == 't1c2':
                            check_c2 = serie_id
                    assert check_pre and check_c1 and check_c2
                    
                    study_dicom_dir = os.path.join(data_basedir, subject, study_id)
                    serie_dir_names = os.listdir(study_dicom_dir)
                    
                    if verbose:
                        print(f"pre: {check_pre}, t1c1: {check_c1}, t1c2: {check_c2}")
                    for serie_dn in serie_dir_names:
                        if serie_dn.startswith(f"{str(check_pre)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)
                            if verbose:
                                print(f"[{subject}/{study_id}] T1pre directory: {full_serie_dir_path}")
                            serie_volume = read_study_dir(full_serie_dir_path)
                            pathlib.Path(os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"))).mkdir(parents=True, exist_ok=True)
                            sitk.WriteImage(serie_volume, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1pre.nii.gz"))
                        elif serie_dn.startswith(f"{str(check_c1)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)
                            if verbose:
                                print(f"[{subject}/{study_id}] T1c1 directory: {full_serie_dir_path}")
                            serie_volume = read_study_dir(full_serie_dir_path)
                            pathlib.Path(os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"))).mkdir(parents=True, exist_ok=True)
                            sitk.WriteImage(serie_volume, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1c1.nii.gz"))
                        elif serie_dn.startswith(f"{str(check_c2)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)
                            if verbose:
                                print(f"[{subject}/{study_id}] T1c2 directory: {full_serie_dir_path}")
                            serie_volume = read_study_dir(full_serie_dir_path)
                            pathlib.Path(os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"))).mkdir(parents=True, exist_ok=True)
                            sitk.WriteImage(serie_volume, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1c2.nii.gz"))
                        else:
                            # not matched
                            continue
                    
                    processed_studies += 1
                elif lat_type == 'multiVolumeBilateral':
                     # Establish series numbers
                    sid_pre = False
                    sid_multivolume_postcontrast = False
                    sid_multivolume_all = False

                    for serie_id, serie_type in series.items():
                        if serie_type == 't1pre':
                            sid_pre = serie_id
                        elif serie_type == 'multiVolumeT1':
                            sid_multivolume_all = serie_id
                        elif serie_type == 'multiVolumeT1Postcontrast':
                            sid_multivolume_postcontrast = serie_id

                    if sid_pre:
                        assert sid_multivolume_all is False  # if pre-c is separate, there cannot be a "Full" multivolume
                        assert sid_multivolume_postcontrast is not False  # if pre-c is separate, there has to be a "post-c" multivolume
                        if verbose:
                            print(f"[{subject}/{study_id}] pre: {sid_pre}; post: {sid_multivolume_postcontrast}")
                    else:
                        print(f"[{subject}/{study_id}] full: {sid_multivolume_all}")

                    study_dicom_dir = os.path.join(data_basedir, subject, study_id)  # boilerplate
                    serie_dir_names = os.listdir(study_dicom_dir)  # boilerplate

                    subvolume_dict = {}
                    for serie_dn in serie_dir_names:
                        if serie_dn.startswith(f"{str(sid_pre)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)
                            print(f"[{subject}/{study_id}] ...multi-volume pre-c")
                            subvolume_dict["pre"] = [os.path.join(full_serie_dir_path, x) for x in sorted(os.listdir(full_serie_dir_path))]

                        elif serie_dn.startswith(f"{str(sid_multivolume_postcontrast)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)
                            print(f"[{subject}/{study_id}] ...multi-volume post-c")
                            subvolume_dict["post"] = process_multivolume(full_serie_dir_path)
                            for post_i, (_, v) in enumerate(subvolume_dict["post"].items()):
                                subvolume_dict[f"post{post_i+1}"] = [os.path.join(full_serie_dir_path, x) for x in v]
                            del subvolume_dict["post"]

                        elif serie_dn.startswith(f"{str(sid_multivolume_all)}.0"):
                            full_serie_dir_path = os.path.join(study_dicom_dir, serie_dn)

                            subvolume_dict = process_multivolume(full_serie_dir_path)
                            print(f"[{subject}/{study_id}] multi-volume processed correctly.")

                    study_paths_dict[study_id] = subvolume_dict

                    if sid_pre:
                        expected_img_num = len(subvolume_dict["pre"])
                        for ss, l in subvolume_dict.items():
                            assert(expected_img_num) == len(l)

                        # Save separate volumes
                        volume_pre = get_volume_given_filepaths(subvolume_dict["pre"])
                        volume_post1 = get_volume_given_filepaths(subvolume_dict["post1"])
                        volume_post2 = get_volume_given_filepaths(subvolume_dict["post2"])
                    else:
                        volume_pre = get_volume_given_filepaths([os.path.join(full_serie_dir_path, x) for x in subvolume_dict["1"]])
                        volume_post1 = get_volume_given_filepaths([os.path.join(full_serie_dir_path, x) for x in subvolume_dict["2"]])
                        volume_post2 = get_volume_given_filepaths([os.path.join(full_serie_dir_path, x) for x in subvolume_dict["3"]])

                    # create directory
                    pathlib.Path(os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"))).mkdir(parents=True, exist_ok=True)

                    # save volume
                    sitk.WriteImage(volume_pre, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1pre.nii.gz"))
                    sitk.WriteImage(volume_post1, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1c1.nii.gz"))
                    sitk.WriteImage(volume_post2, os.path.join(save_basedir, f"{subject}___{study_id}".replace(" ", "_"), f"{serie_id}-t1c2.nii.gz"))

                    processed_studies += 1
                elif lat_type in ['unilateralLeft', 'unilateralRight', 'multiVolumeUnilateralLeft', 'multiVolumeUnilateralRight']:
                    print(f"[{subject}/{study_id}] EXCLUDED because of lat type {lat_type}. (This script excludes unilateral exams)")
        print(processed_studies)

### 1. Download the dataset
You can download the TCGA-BRCA dataset from The Cancer Imaging Archive (TCIA) website: https://wiki.cancerimagingarchive.net/display/Public/TCGA-BRCA

Make sure that you download both imaging files as well as the Clinical Data zip file.

### 2. Provide paths to downloaded files

In [ ]:
# Provide path where your TCGA-BRCA dataset is downloaded
base_dir = "/path/to/your/tcgabrca/dataset/TCGA-BRCA/"

# Please find file named `nationwidechildrens.org_clinical_patient_brca.txt`
# and provide path to the file below:
clinical_file = "/path/to/clinical/file/nationwidechildrens.org_clinical_patient_brca.txt"

# Provide path to a directory where you want to save 
# preprocessed images
output_dir = "/path/where/to/save/output/"

In [ ]:
# Load the file that identifies all pre- and post-contrast series types
with open("tcga_brca_key.yaml", "r") as f:
    pre_post_dict = yaml.safe_load(f)

### 3. Check completeness of download

In [ ]:
print("Complete download should consist of 139 subjects.")
print(f"Found {len(os.listdir(base_dir))} subjects.")

### 4. Pre-process downloaded data

In [ ]:
preprocess_tcga_brca(
    pre_post_dict, 
    data_basedir=base_dir,
    save_basedir=output_dir,
    dry_run=False, 
    verbose=True
)

### 5. Generate a datalist for training
This part generates a DataFrame with paths to your nifti images and correct labels

In [ ]:
new_datalist = {}
df = pd.read_csv(clinical_file, delimiter="\t")
df = df[['bcr_patient_barcode', 'patient_id', 'anatomic_neoplasm_subdivision', 
    'menopause_status', 'ajcc_tumor_pathologic_pt', 'ajcc_nodes_pathologic_pn',
    'ajcc_metastasis_pathologic_pm', 'er_status_by_ihc', 'er_status_ihc_Percent_Positive', 'er_ihc_score',
    'pr_positivity_ihc_intensity_score', 'her2_ihc_score']][2:]

for study_dir in sorted(os.listdir(output_dir)):
    new_datalist[study_dir] = {
        "label": None,
        "path_pre": None,
        "path_post1": None,
        "path_post2": None        
    }
    
    # Store paths
    for f in os.listdir(os.path.join(output_dir, study_dir)):
        if "t1pre" in f:
            new_datalist[study_dir]['path_pre'] = os.path.join(output_dir, study_dir, f)
        elif "t1c1" in f:
            new_datalist[study_dir]['path_post1'] = os.path.join(output_dir, study_dir, f)
        elif "t1c2" in f:
            new_datalist[study_dir]['path_post2'] = os.path.join(output_dir, study_dir, f)
    
    # Store labels
    for key in new_datalist.keys():
        patient_id = key[8:12]
        df_x = df[df.patient_id == patient_id]
        anatomic_loc = df_x.iloc[0].anatomic_neoplasm_subdivision
        if "left" in anatomic_loc.lower():
            new_datalist[key]['label'] = [0, 1, 0, 0]
        elif "right" in anatomic_loc.lower():
            new_datalist[key]['label'] = [0, 0, 0, 1]
        else:
            raise ValueError()

In [ ]:
datalist_df = pd.DataFrame.from_dict(new_datalist, orient='index')

In [ ]:
datalist_df